## Datasets creation

In [ ]:
## Reference
# Global Stock Market Prediction Based on Stock Chart Images Using Deep Q-Network,
# Authors: Jinho Lee, Raehyun Kim, Yookyung Koh, and Jaewoo Kang,
# URL https://arxiv.org/abs/1902.10948

#Obtain SP500 tickers
import urllib.request
from html_table_parser import HTMLTableParser

url_snp500 = 'http://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

def obtain_parse_wiki_stocks_sp500(url):
    """Download and parse the Wikipedia list of S&P500 
    constituents using requests and libxml.

    Returns a list of tuples for to add to MySQL."""

    # Get S&P500 website content
    req = urllib.request.Request(url)
    response = urllib.request.urlopen(req)
    data = response.read().decode('utf-8')

    #Instantiate the parser and feed it
    p = HTMLTableParser()
    p.feed(data)
    table_list = p.tables
    table = table_list[0][1:]

    # Obtain the symbol information for each row in the S&P500 constituent table
    symbols = []
    for row in table:
        sd = {'ticker': row[0],
            'name': row[1],
            'sector': row[3]}
        # Create a tuple (for the DB format) and append to the grand list
        symbols.append(sd['ticker'])
    return symbols

snp500_tickers = obtain_parse_wiki_stocks_sp500(url_snp500)

In [ ]:
companies_bundles = list()
for i in range(0,len(snp500_tickers),5):
    companies_bundles.append(snp500_tickers[i:i+5])

In [ ]:
import os
from pandas_datareader import data as pdr
import yfinance as yf
import pandas as pd
import datetime
import numpy as np
import time

yf.pdr_override()

In [ ]:
base_path = r'YourSP500DataPath'

def build_ticker_dataset(path, ticker, start, end):
    """
    Creates the dataset containing all stock prices
    :returns: stock_prices.csv
    """

    # Get all Adjusted Close prices for all the tickers in our list,
    # between START_DATE and END_DATE
    data = pdr.get_data_yahoo(ticker, start, end)
    if len(data.dropna())>0.75*len(data):
        data.to_csv(os.path.join(path,ticker+'.csv'))

def quantilize(df):
    #Find the 1st and the 3rd of each column Q1 and Q3
    df_quantiles = df.quantile([0.25,0.75])
    
    #Adjust(symmetrize) columns distribution using the interval
    #[Q1-1.5*IQR,Q3+1.5*IQR] where IQR = Q3 - Q1
    
    df_adj = pd.DataFrame(columns = df.columns, index = df.index)
    df_adj['Date'] = df['Date']
    
    cols = df_quantiles.columns
    for col in cols:
        Q1 = df_quantiles.loc[0.25][col]
        Q3 = df_quantiles.loc[0.75][col]
        IQR = Q3 - Q1
        B1 = Q1 - 2.5*IQR ; B2 = Q3 + 2.5*IQR
        
        df_adj[col] = np.where((df[col] > B1) & (df[col] < B2), df[col], np.where((df[col] < B1), B1, B2))
    df_adj['Volume'] = df_adj['Volume'].astype(np.int64)
    
    return df_adj

In [ ]:
country_ticker = 'US'
start_date = "2000-01-01"
end_date = "2020-12-31"

true_companies_bundles = list()
for companies_bundle in companies_bundles:
    for ticker in companies_bundle:
        print(ticker)
        path = os.path.join(base_path,country_ticker)
        concatdates = start_date[:4]+'_'+str(int(end_date[:4])+1)

        dir_path1 = os.path.join(path,'W_'+concatdates)
        if not os.path.exists(dir_path1):
            os.makedirs(dir_path1)   
        build_ticker_dataset(dir_path1, ticker, start_date, end_date)
    
    dictlens = dict()
    for ticker in companies_bundle:
        try:
            df = pd.read_csv(os.path.join(dir_path1, ticker+'.csv'))
            l = len(df.dropna())
            dictlens[l] = dictlens.get(l,0)+1
        except FileNotFoundError:
            companies_bundle.remove(ticker)

    mostrecurrentlen = 0
    numbrecs = 0
    for l in dictlens.keys():
        if dictlens[l] >numbrecs:
            numbrecs = dictlens[l]
            mostrecurrentlen = l
    
    tickers_to_consider = list()
    for ticker in companies_bundle:
        df = pd.read_csv(os.path.join(dir_path1, ticker+'.csv'))
        if len(df.dropna())==mostrecurrentlen:
            tickers_to_consider.append(ticker)
    if len(tickers_to_consider)>1:
        true_companies_bundles.append(tickers_to_consider)

In [ ]:
for companies_bundle in true_companies_bundles:
    print('')
    print('Companies bundle {} started'.format(companies_bundle))
    for ticker in companies_bundle:

        #Split train_test
        ticker0 = ticker+'.csv'
        df = pd.read_csv(os.path.join(dir_path1, ticker0))
        df = df.dropna()
        n = int(0.0*len(df))
        l = int(0.8*len(df))
        f = int(0.2*len(df))
        df_train, df_test = df[n:n+l], df[n+l:n+l+f]

        #Train
        start_train_date = list(df_train['Date'])[0]
        end_train_date = list(df_train['Date'])[-1]

        concatdates = start_train_date[:4]+'_'+str(int(end_train_date[:4]))
        dir_path2 = os.path.join(path,'Train','W_'+concatdates)

        if not os.path.exists(dir_path2):
            os.makedirs(dir_path2)   

        df_train.to_csv(os.path.join(dir_path2,ticker0))

        #Test
        start_test_date = list(df_test['Date'])[0]
        end_test_date = list(df_test['Date'])[-1]

        concatdates = start_test_date[:4]+'_'+str(int(end_test_date[:4]))
        dir_path3 = os.path.join(path,'Test','W_'+concatdates)

        if not os.path.exists(dir_path3):
            os.makedirs(dir_path3)   

        df_test.to_csv(os.path.join(dir_path3,ticker0))
                
    print('')
    print('Companies bundle {} completed'.format(companies_bundle))
        